# Python でのデータ分析の基礎

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
%matplotlib inline

## データの読み込み

あるレストランのウェイターが数ヶ月の期間に受け取ったチップについて記録したデータをデータフレームとして読み込ます。結果を `tips` と名付けます。データは244行、7列からなります。各列は

- bill: 請求額
- tip: チップの額
- sex: 会計をした客の性別
- smoker: 客が喫煙者か否か
- day: 週の曜日
- time: 時間帯 ('Lunch', 'Dinner')
- size: パーティーのサイズ

で、金額の単位はドルです。

In [ ]:
tips = pd.read_csv('data/tips.csv')

データの行数と列数を確認します。

In [ ]:
tips.shape

上部と下部のレコードを確認します。

In [ ]:
tips.head()

In [ ]:
tips.tail()

列名を確認します。

In [ ]:
tips.columns

各列のデータ型を確認します。文字列は `object` と表示されます。

In [ ]:
tips.dtypes

要素統計量を確認します。

In [ ]:
tips.describe()

数値型ではない列の要素統計量の確認をするには `describe(include=['O'])` とします。

In [ ]:
tips.describe(include=['O'])

## データの選択

列 tip を取り出します。

In [ ]:
tips['tip'].head()

列 tip と day を取り出します。

In [ ]:
tips[['tip', 'day']].head()

5 から 9 番目のレコードを取り出します。

In [ ]:
tips[5:10]

チップの額が 7 ドルより大きいレコードを取り出します。

In [ ]:
tips[tips['tip'] > 7]

チップの額が 4 ドル以上 5 ドル未満のレコードを取り出します。

In [ ]:
tips[(tips['tip'] >= 4) & (tips['tip'] < 5)].head()

上記のような列の値による選択は `query()` で簡単に処理することができます。

In [ ]:
tips.query('tip >= 4 & tip < 5').head()

## 欠損データの確認と対処

欠損データを含むデータファイルを読み込みます。

In [ ]:
tips = pd.read_csv('data/tips-2.csv')

データフレームの上部を確認します。

In [ ]:
tips.head()

各列のデータ型を確認します。

In [ ]:
tips.dtypes

int 型であるはずの列 size が float 型になっています。これは列の値に欠損データを含む可能性があることを示唆しています。欠損を一つでも含む列を確認します。

In [ ]:
tips[tips.isnull().any(axis=1)]

`bill` と `tip` のデータ欠損を削除します。

In [ ]:
tmp_1 = tips.dropna(subset=['bill', 'tip'])

In [ ]:
tmp_1.dtypes

正しく削除できたかを確認します。

In [ ]:
tmp_1[tmp_1.isnull().any(axis=1)]

`size` のデータ欠損は `size` の最頻値で修正します。`size` の各値に対応するレコード数を確認します。

In [ ]:
tips['size'].value_counts()

最頻値は 2 であることがわかります。size のデータ欠損を 2 で修正します。

In [ ]:
tmp_2 = tmp_1.fillna({'size': 2})

正しく修正できたかを確認します。

In [ ]:
tmp_2[tmp_2.isnull().any(axis=1)]

In [ ]:
tips = tmp_2

列 `size` のデータ型を int 型に変換します。

In [ ]:
tips['size'] = tips['size'].astype('int')

## グルーピング

週の曜日ごとの要素統計量を知りたくなりました。このように、ある列の値ごとの何かを集計したい場合はグループバイという処理を用います。 

グループバイは以下の一連のステップからなる処理のことです。

1. データをある条件によってグループに分割する
2. それぞれのグループに対して独立に関数を適用する（例えば、総和をとる、平均をとる等）
3. 結果を結合してデータフレームにする

データを `day` の値によってグループに分割します。グルーピングの結果は `DataFrameGroupBy` オブジェクトです。

In [ ]:
grouped = tips.groupby('day')

In [ ]:
type(grouped)

どのようにグループに分割されたか確認してみます。

In [ ]:
for name, group in grouped:
    print(name)
    print(group)
    print()

列 `day` の値によってグループに分割されていることが確認できました。`describe()` を用いてそれぞれのグループの要素統計量を集計します。

In [ ]:
grouped.describe()

出力が横に長過ぎて見づらいので行と列を転置します。

In [ ]:
grouped.describe().transpose()

In [ ]:
grouped['tip'].sum()

## 集計と可視化

便宜上、文字列型のデータをカテゴリー型に変換します。

In [ ]:
categories = {
    'day': ['Thur', 'Fri', 'Sat', 'Sun'],
    'sex': ['Male', 'Female'],
    'time': ['Lunch', 'Dinner'],
    'smoker': ['Yes', 'No']
}
for col, values in categories.items():
    tips[col] = pd.Categorical(tips[col], values)

In [ ]:
tips.dtypes

このウェイターが集計期間にチップだけでいくら稼いだかを計算してみましょう。

In [ ]:
tips['tip'].sum()

728ドル！結構稼いでいますね。ディナータイムとランチタイムでは、どちらでよりチップを稼いでいたのかを見てみましょう。`groupby()` で時間帯ごとにグルーピングして `tip` の総和を計算します。

In [ ]:
tips.groupby('time')['tip'].sum()

圧倒的にディナータイムにチップを稼いでいたようですね。

チップの額の上位10のレコードを調べてみましょう。

In [ ]:
tips.sort_values(by='tip', ascending=False).head(10)

このレストランの客の喫煙者と非喫煙者の割合はそれくらいでしょうか？分かりやすいように円グラフとしてプロットしてみます。`groupby()` で喫煙者・非喫煙者でグルーピングしてから列 `tip` レコード数を取り出します。結果を `smoker` と名付けます。

In [ ]:
smoker = tips.groupby('smoker')['tip'].count()

`smoker` を円グラフとしてプロットします。

In [ ]:
smoker.plot.pie(figsize=(6, 6))

喫煙者と非喫煙者の割合はおよそ 2:1 というところでしょうか。

週の曜日ごとに請求額とチップの額の平均を棒グラフとしてプロットします。`groupby()` で曜日ごとにグルーピングして `bill` と `tip` の平均をとります。結果を `tips_by_dow` と名付けます。

In [ ]:
tips_by_dow = tips.groupby('day')[['bill', 'tip']].mean()
tips_by_dow.head()

`tips_by_dow` を棒グラフとしてプロットします。

In [ ]:
tips_by_dow.plot.bar()

週日にくらべて週末のほうが請求額が多くなってることがわかります。チップの額は微増しているように見えます。上のグラフでは、支払い額がひと目ではわかりにくくなっています。`tips_by_dow` を積み上げ棒グラフとしてプロットします。

In [ ]:
tips_by_dow.plot.bar(stacked=True)

週日にくらべて週末のほうが支払額が多くなってることがひと目でわかります。しかし、チップの額の変化は上のグラフの方が見やすいですね。支払額におけるチップの額の割合を知りたくなりました。支払額を 1 とした場合のチップの額の割合を計算します。結果を `frac_tips_by_dow` と名付けます。この計算には `apply()` を用います。デフォルトでは `apply()` は引数として渡した関数を各列に適用します。

In [ ]:
frac_tips_by_dow = tips_by_dow.apply(lambda x : x / tips_by_dow.sum(axis=1))
frac_tips_by_dow

支払額を 1 とした場合のチップの額の割合を曜日ごとに積み上げ棒グラフとしてプロットします。

In [ ]:
frac_tips_by_dow.plot.bar(stacked=True)

支払額におけるチップの額の割合は、ほとんど変化していませんが週末に微減していることがわかりました。

上の一連のグラフのY軸にはラベルがありません。また、X軸のラベルが day というのは分かりやすい名前とは言えません。さらに、上のグラフでは凡例が棒グラフを重なってしまって見えにくくなっています。グラフの見た目を整えるときには [matplotlib](https://matplotlib.org/) を用います。

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
(tips_by_dow
 .apply(lambda x : x / tips_by_dow.sum(axis=1))
 .plot.bar(stacked=True, ax=ax))
ax.set_xlabel('Day of week')
ax.set_ylabel('Bill vs tip')
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

支払額の棒グラフと支払額におけるチップの額の割合の棒グラフを横に並べてプロットしてみます。

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

tips_by_dow.plot.bar(stacked=True, ax=axes[0])
(tips_by_dow
 .apply(lambda x : x / tips_by_dow.sum(axis=1))
 .plot.bar(stacked=True, ax=axes[1]))

for ax in axes:
    ax.set_xlabel('Day of week')
axes[0].set_ylabel('Total payment')
axes[1].set_ylabel('Bill vs tip')
axes[0].legend().set_visible(False)
axes[1].legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

### 練習問題

In [ ]:
# 時間ごと（Lunch/Dinner）の請求額とチップの額の平均を可視化してください。何がわかりますか？